In [58]:
import pandas as pd
from pymongo import MongoClient

In [59]:
uri = "mongodb://localhost:27017"
db_name = "mtg_decks_db"
client = MongoClient(uri)
db = client[db_name]

collection = db["personal_pool"]

In [60]:
df = pd.read_csv("../raw-data/moxfield_wishlist.csv")
df.head()

,Count,Name
0,3,Agony Warp
1,4,Alms of the Vein
2,4,Ancient Grudge
3,4,Annoyed Altisaur
4,3,Arashin Cleric


In [61]:
df = df[["Count", "Name"]]
df = df.rename(columns={"Count": "count", "Name": "name"})

df.head()

,count,name
0,3,Agony Warp
1,4,Alms of the Vein
2,4,Ancient Grudge
3,4,Annoyed Altisaur
4,3,Arashin Cleric


In [62]:
selected_columns = df[["count", "name"]]
cards = selected_columns.to_dict(orient="records")
cards

[{'count': 3, 'name': 'Agony Warp'},
 {'count': 4, 'name': 'Alms of the Vein'},
 {'count': 4, 'name': 'Ancient Grudge'},
 {'count': 4, 'name': 'Annoyed Altisaur'},
 {'count': 3, 'name': 'Arashin Cleric'},
 {'count': 4, 'name': 'Arbor Elf'},
 {'count': 4, 'name': 'Ardent Recruit'},
 {'count': 4, 'name': 'Avenging Hunter'},
 {'count': 4, 'name': 'Axebane Guardian'},
 {'count': 4, 'name': 'Azorius Guildgate'},
 {'count': 4, 'name': 'Barbed Batterfist'},
 {'count': 4, 'name': 'Basilisk Gate'},
 {'count': 4, 'name': 'Battle Screech'},
 {'count': 4, 'name': 'Behold the Multiverse'},
 {'count': 2, 'name': 'Benevolent Bodyguard'},
 {'count': 4, 'name': 'Blanchwood Prowler'},
 {'count': 4, 'name': 'Boarding Party'},
 {'count': 4, 'name': 'Boros Garrison'},
 {'count': 4, 'name': 'Campfire'},
 {'count': 4, 'name': 'Candy Trail'},
 {'count': 4, 'name': "Chainer's Edict"},
 {'count': 4, 'name': 'Circle of Protection: Blue'},
 {'count': 4, 'name': 'Citadel Gate'},
 {'count': 4, 'name': 'Cleansing Wi

In [63]:
import_type = "sum"  # sum or replace


def add_or_update_card(card):
    card_from_pool = db["pool"].find_one({"scryfall_name": card["name"]})
    name = card_from_pool["name"] if card_from_pool else card["name"]

    existing_card = collection.find_one({"scryfall_name": card["name"]})
    if not existing_card:
        collection.insert_one(
            {"name": name, "scryfall_name": card["name"], "count": card["count"]})
        return

    if import_type == "sum":
        card["count"] += existing_card["count"]

    collection.update_one({"_id": existing_card["_id"]}, {
                          "$set": {"count": card["count"]}})

In [64]:
for card in cards:
    add_or_update_card(card)